In [ ]:
import sys
import os 
sys.path.append(os.path.abspath(os.path.join("", "..")))
import torch
import warnings
warnings.filterwarnings("ignore")
from PIL import Image
from lora_w2w import LoRAw2w, LoRAModule
from utils import load_models, inference, save_model_w2w, save_model_for_diffusers
from inversion_utils_experimental import invert
device = "cuda:0"

unet, vae, text_encoder, tokenizer, noise_scheduler = load_models(device)

In [ ]:
mean = torch.load("../files/mean.pt").bfloat16().to(device)
std = torch.load("../files/std.pt").bfloat16().to(device)
v = torch.load("../files/V.pt").bfloat16().to(device)
weight_dimensions = torch.load("../files/weight_dimensions.pt")
proj = torch.zeros(1,10000).bfloat16().to(device)
network = LoRAw2w( proj, mean, std, v[:, :10000], 
                    unet,
                    rank=1,
                    multiplier=1.0,
                    alpha=27.0,
                    train_method="xattn-strict"
                ).to(device, torch.bfloat16)

In [ ]:
prompt = "sks person"

network = invert(network=network, unet=unet, vae=vae, text_encoder=text_encoder, tokenizer=tokenizer, 
                 prompt=prompt, noise_scheduler = noise_scheduler, epochs=100, 
                 image_path = "../inversion/images/deepfake", 
                 mask_path = "../inversion/images/deepfake_masks", device = device, 
                 weight_decay=0)

In [ ]:
negative_prompt = "low quality, blurry, unfinished" 
batch_size = 1
height = 512
width = 512
guidance_scale = 6
seed = 2
ddim_steps = 50

import random

seeds = [random.randint(1, 100000) for _ in range(4)] 


for i in range(4):
    generator = torch.Generator(device=device).manual_seed(seeds[i])
    
    image = inference(network, unet, vae, text_encoder, tokenizer, prompt, negative_prompt, guidance_scale, noise_scheduler, ddim_steps, seeds[i], generator, device)

    image = image.detach().cpu().float().permute(0, 2, 3, 1).numpy()[0]
    image = Image.fromarray((image * 255).round().astype("uint8"))
    image.save(f"output/img_{i}.png")

In [ ]:
save_model_w2w(network, path="../output/deepfake10000")